In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import importlib

from datetime import datetime
from google.cloud.bigquery import Client, LoadJobConfig

sys.path.append('../lib')
import utils
import pandas_utils
import pesys_utils

importlib.reload(utils)
importlib.reload(pandas_utils)
importlib.reload(pesys_utils)

utils.set_secrets_as_env()

# Antall krav, førstegangsbehandling innland/utland

In [ ]:
con = pesys_utils.open_pen_connection()
df_krav = pandas_utils.pandas_from_sql('../sql/kravcount.sql', con)
con.close() 

In [ ]:
df = df_krav.copy()
#df["ANTALL"] = df.ANTALL.apply(lambda x: utils.randomize_zeros(x)) # Er dette lurt?

In [ ]:
df_forstegang = df[df.KRAVTYPE.isin(pesys_utils.forstegang)]
df_forstegang.head()

In [ ]:
table_id = f'pensjon-saksbehandli-prod-1f83.saksstatistikk.krav_forstegang'
job_config = LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client = Client(project="pensjon-saksbehandli-prod-1f83")

job = client.load_table_from_dataframe(df_forstegang, table_id, job_config=job_config)
job.result()

## Alle krav, med filter på om det kommer fra bruker

In [ ]:
df = df_krav.copy()

In [ ]:
df.head()

df["FRA_BRUKER"] = df.KRAVTYPE.isin(kravtyper)

## Lag en fil med alle kravtyper og hvor mange det har vært av dem

In [ ]:
# con = pesys_utils.open_pen_connection()

# df_tmp = pandas_utils.pandas_from_sql('../sql/kravtyper_antall.sql', con)
# con.close()

In [ ]:
#df_tmp.sort_values("ANTALL", ascending=False).reset_index(drop=True).to_csv("k_krav_gjelder i pesys.csv")

# Ledetid

In [ ]:
con = pesys_utils.open_pen_connection()

df_ledetid = pandas_utils.pandas_from_sql('../sql/ledetid_grouped.sql', con, tuning=200000)
con.close()

In [ ]:
df.head()

# Klager

In [ ]:
con = pesys_utils.open_pen_connection()

df_klager = pandas_utils.pandas_from_sql('../sql/GP_BP/klager.sql', con)
con.close()

In [ ]:
df = df_klager.copy()
df["AR"] = df["AR"].apply(str)

In [ ]:
df_bp = df[df.STOENAD == "Barnepensjon"]
df_gjenlev = df[df.STOENAD == "Gjenlevendeytelse"]

In [ ]:
subset = {"Stadfestet", "Omgjøre", "Delvis omgjøre", "Avvist", "Opphevet og hjemvist"}
group_color = {key: theme.colors.navColors[i] for i, key in enumerate(subset)}
labels["ANTALL"] = "Antall vedtak"